In [1]:
from utils.functions import *
from datasets import Dataset

%load_ext autoreload
%autoreload 2

/opt/mamba/envs/putain_de_projet_nlp/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729138878/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


update saved


In [2]:
model = transformers.AutoModelForSequenceClassification.from_pretrained(
    "textattack/bert-base-uncased-ag-news"
)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    "textattack/bert-base-uncased-ag-news"
)

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)
goal_function = UntargetedClassification(model_wrapper)
dataset = HuggingFaceDataset("ag_news", None, "test")

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/home/onyxia/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


Separating train and test data

In [3]:
normal_text = pd.read_csv(r"data/attacks/pwws.csv").original_text
df_attacks = pd.concat(
    [
        pd.read_csv(r"data/attacks/pwws.csv"),
        pd.read_csv(r"data/attacks/bae.csv"),
        pd.read_csv(r"data/attacks/textfooler.csv"),
        pd.read_csv(r"data/attacks/tf-adj.csv"),
    ]
)
attacks = df_attacks.apply(
    lambda x: x.perturbed_text if x.result_type != "Skipped" else None, axis=1
).dropna()
labels = [0] * len(normal_text) + [1] * len(attacks)
data = pd.DataFrame({"text": [*normal_text, *attacks], "label": labels})

In [4]:
n = len(data)
windows = np.linspace(0, n, 100).astype(int)
for batch_idx in tqdm(range(len(windows) - 1)):
    batch_data = data[windows[batch_idx] : windows[batch_idx + 1]]
    batch_data = Dataset.from_pandas(batch_data)
    batch_data_loader = torch.utils.data.DataLoader(
        batch_data, batch_size=32, shuffle=False
    )
    last_layer, softmax = run_model(batch_data_loader, output_hs=True)
    torch.save(
        last_layer, r"data//BERT embedding//last_layer_n_vs_a " + str(batch_idx + 1)
    )
    torch.save(softmax, r"data//BERT embedding//softmax_n_vs_a " + str(batch_idx + 1))

100%|██████████| 57/57 [1:41:25<00:00, 106.77s/it]


In [10]:
softmax = torch.cat(
    [
        torch.load(r"data//BERT embedding//softmax_n_vs_a " + str(idx + 1))
        for idx in range(99)
    ]
)
torch.save(softmax, r"data//BERT embedding//softmax_n_vs_a")
last_layer = torch.cat(
    [
        torch.load(r"data//BERT embedding//last_layer_n_vs_a " + str(idx + 1))
        for idx in range(99)
    ]
)
torch.save(last_layer, r"data//BERT embedding//last_layer_n_vs_a")

In [24]:
data.to_csv(r"data//BERT embedding//full_dataset.csv")